In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV,
    GridSearchCV
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Imbalanced-learn
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline


In [9]:
"/Users/maximbortnik/Downloads/creditcard.csv"
df = pd.read_csv("/Users/maximbortnik/Downloads/creditcard.csv")

print("Размер исходного набора данных:", df.shape)
df.head()


Размер исходного набора данных: (284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
missing = df.isnull().sum()
print(missing[missing > 0])

Series([], dtype: int64)


In [11]:
# Перемешивание данных
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [12]:
# Разделение данных
X = df.drop("Class", axis=1)
y = df["Class"]

X_full_train, X_test, y_full_train, y_test = train_test_split(
    X, y, 
    stratify=y,
    test_size=0.2, 
    random_state=42
)

print("Train size:", X_full_train.shape)
print("Test size: ", X_test.shape)


Train size: (227845, 30)
Test size:  (56962, 30)


In [13]:
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(
        random_state=42, 
    ))
])

In [14]:
X_tune, _, y_tune, _ = train_test_split(
    X_full_train, 
    y_full_train,
    train_size=0.3,
    stratify=y_full_train,
    random_state=42
)

print("Выборка для подбора гиперпараметров:", X_tune.shape)


Выборка для подбора гиперпараметров: (68353, 30)


In [ ]:
param_distributions = {
    "clf__n_estimators": [50, 100, 200],
    "clf__max_depth": [5, 10, 20, None], 
    "clf__min_samples_leaf": [1, 2, 5],
    "clf__class_weight": ["balanced", "balanced_subsample", None]
}

from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=10,            
    scoring="f1",         
    cv=3,                  
    verbose=1,
    random_state=42,
    n_jobs=-1            
)

random_search.fit(X_tune, y_tune)
print("Лучшие параметры:", random_search.best_params_)
print("Лучший F1 (CV):   ", random_search.best_score_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [11]:
best_params = random_search.best_params_
print("Параметры для финального обучения:", best_params)

final_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(
        n_estimators=best_params["clf__n_estimators"],
        max_depth=best_params["clf__max_depth"],
        min_samples_leaf=best_params["clf__min_samples_leaf"],
        class_weight=best_params["clf__class_weight"],
        random_state=42
    ))
])

# Обучаем на полной train-выборке
final_pipeline.fit(X_full_train, y_full_train)


Параметры для финального обучения: {'clf__n_estimators': 200, 'clf__min_samples_leaf': 1, 'clf__max_depth': 10, 'clf__class_weight': None}


Pipeline(steps=[('scaler', StandardScaler()),
                ('clf',
                 RandomForestClassifier(max_depth=10, n_estimators=200,
                                        random_state=42))])

In [12]:
y_pred = final_pipeline.predict(X_test)

# Метрики
print("== Confusion matrix ==")
print(confusion_matrix(y_test, y_pred))

print("\n== Classification report ==")
print(classification_report(y_test, y_pred, digits=4))

y_probas = final_pipeline.predict_proba(X_test)[:, 1]
test_roc_auc = roc_auc_score(y_test, y_probas)
print("ROC-AUC на тесте:", test_roc_auc)


== Confusion matrix ==
[[56859     5]
 [   26    72]]

== Classification report ==
              precision    recall  f1-score   support

           0     0.9995    0.9999    0.9997     56864
           1     0.9351    0.7347    0.8229        98

    accuracy                         0.9995     56962
   macro avg     0.9673    0.8673    0.9113     56962
weighted avg     0.9994    0.9995    0.9994     56962

ROC-AUC на тесте: 0.9663465748567294


Ключевые моменты:

1) Высокая точность (Precision) по мошенничеству (93.51%) — то есть когда модель говорила «мошенничество», она почти всегда была права, ложных тревог было очень мало (FP=5).
2) Recall = 73.47% — значит, ~26.5% реальных мошенничеств модель пропускала (FN=26).
3) ROC-AUC = 0.9663 — высокий показатель, модель в среднем неплохо различала классы.

In [ ]:
# Визуализация матрицы ошибок
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Not Fraud", "Fraud"], yticklabels=["Not Fraud", "Fraud"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

Применим SMOTE


In [14]:
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("smote", SMOTE(random_state=42)),
    ("clf", RandomForestClassifier(random_state=42))
])

In [15]:
param_distributions = {
    "clf__n_estimators": [50, 100, 200],
    "clf__max_depth": [5, 10, 20, None],
    "clf__min_samples_leaf": [1, 2, 5],
    "clf__class_weight": [None, "balanced", "balanced_subsample"]
}

In [16]:
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    scoring="f1",
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1  
)
random_search.fit(X_tune, y_tune)

print("Лучшие параметры:", random_search.best_params_)
print("Лучший F1 (CV):  ", random_search.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Лучшие параметры: {'clf__n_estimators': 200, 'clf__min_samples_leaf': 5, 'clf__max_depth': 20, 'clf__class_weight': 'balanced'}
Лучший F1 (CV):   0.8345220046231091


In [17]:
best_params = random_search.best_params_
final_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("smote", SMOTE(random_state=42)),
    ("clf", RandomForestClassifier(
        n_estimators=best_params["clf__n_estimators"],
        max_depth=best_params["clf__max_depth"],
        min_samples_leaf=best_params["clf__min_samples_leaf"],
        class_weight=best_params["clf__class_weight"],
        random_state=42
    ))
])
final_pipeline.fit(X_full_train, y_full_train)

Pipeline(steps=[('scaler', StandardScaler()), ('smote', SMOTE(random_state=42)),
                ('clf',
                 RandomForestClassifier(class_weight='balanced', max_depth=20,
                                        min_samples_leaf=5, n_estimators=200,
                                        random_state=42))])

In [18]:
y_pred = final_pipeline.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("\n== Confusion Matrix ==")
print(cm)

print("\n== Classification Report ==")
print(classification_report(y_test, y_pred, digits=4))

y_proba = final_pipeline.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_proba)
print("ROC-AUC:", roc_auc)


== Confusion Matrix ==
[[56846    18]
 [   18    80]]

== Classification Report ==
              precision    recall  f1-score   support

           0     0.9997    0.9997    0.9997     56864
           1     0.8163    0.8163    0.8163        98

    accuracy                         0.9994     56962
   macro avg     0.9080    0.9080    0.9080     56962
weighted avg     0.9994    0.9994    0.9994     56962

ROC-AUC: 0.9795429373916139


Ключевые моменты:

1) Recall (81.63%) ощутимо вырос по сравнению с 73.47%: модель стала пропускать меньше мошенничеств (FN=18 вместо 26).
2) При этом Precision (81.63%) заметно снизился относительно предыдущего (93.51%): выросло число ложных тревог (FP=18 вместо 5).
3) ROC-AUC (0.9795) даже выше, чем было (0.9663). Это указывает, что в среднем ранжирование вероятностей улучшилось.

Сравнение двух подходов

**Recall:**

Был 73.47%, стал 81.63% — лучше ловим мошенничество (уменьшили пропуски с 26 до 18).

**Precision:**

Было 93.51%, стало 81.63% — то есть модель теперь чаще «зря бьёт тревогу» (FP выросли с 5 до 18).
Это ожидаемое изменение, так как, чтобы ловить больше фрода, модель вынуждена рисковать большим числом ложных срабатываний.

**F1-score** (для класса 1)

Первая модель: 0.8229
Вторая (SMOTE): 0.8163
F1 немного снизился, но Recall вырос, Precision упал — итоговый баланс оказался чуть меньшим, чем был (хотя разница не очень большая).

**ROC-AUC**

Было 0.9663, стало 0.9795 — прирост. 
Повышение ROC-AUC свидетельствует, что по среднему ранжированию вероятностей новая модель отрабатывает лучше.